In [1]:
#Importing Packages
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from copy import deepcopy
from IPython.display import clear_output
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
#Importing dataset
data=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')
data=data[['title','genres','overview']]
fortopicdf = data.copy()

FileNotFoundError: ignored

In [ ]:
fortopicdf['overview']=fortopicdf['overview'].fillna('')
fortopicdf

In [ ]:
data.head()

In [ ]:
def make_it_neat(data,name):
    arr=data[name].values
    pp=[]
    for i in arr:
        x=i
        x=x.replace('{','').replace('"id"','').replace('"name"','').replace('}','').replace('[','').replace(']','').replace(':','').replace('"','')
        p=[]
        for i in x.split(','):
            if i[2:].isdigit():
                continue
            else:
                p.append(i[2:])
        pp.append(p)
    data[name]=pp
    
make_it_neat(data,'genres')

In [ ]:
data.head()

In [ ]:
data['genres'].values[0]

In [ ]:
# Filling all the empty/nan description rows with empty string
data['overview']=data['overview'].fillna('')

In [ ]:
# This function is to remove stopwords from a particular column and to tokenize it
def rem_stopwords_tokenize(data,name):
      
    def getting(sen):
        example_sent = sen

        stop_words = set(stopwords.words('english')) 

        word_tokens = word_tokenize(example_sent) 

        filtered_sentence = [w for w in word_tokens if not w in stop_words] 

        filtered_sentence = [] 

        for w in word_tokens: 
            if w not in stop_words: 
                filtered_sentence.append(w) 
        return filtered_sentence
    x=[]
    for i in data[name].values:
        x.append(getting(i))
    data[name]=x

rem_stopwords_tokenize(data,'overview')

In [ ]:
# Making a function to lemmatize all the words
lemmatizer = WordNetLemmatizer() 
def lemmatize_all(data,name):
    arr=data[name]
    a=[]
    for i in arr:
        b=[]
        for j in i:
            x=lemmatizer.lemmatize(j,pos='a')
            x=lemmatizer.lemmatize(x)
            b.append(x)
        a.append(b)
    data[name]=a

    
lemmatize_all(data,'overview')

In [ ]:
dic_genres={}
dic_overview={}
for i in data.genres:
    for j in i:
        if j not in dic_genres:
            dic_genres[j]=0
        else:
            continue
for i in data.overview:
    for j in i:
        if j not in dic_overview:
            dic_overview[j]=0
        else:
            continue

In [ ]:
df=deepcopy(data)

In [ ]:
# This function is made to convert words to vector
def vectorizer(data,name,d):
    arr=data[name].values
    pp=[]
    count=0
    l=len(df)
    for i in arr:
        count+=1
        clear_output(wait=True)
        print('The progress is:','{:.2f}'.format(count*100/l),' %')
        dic=deepcopy(d)
        p=[]
        for j in i:
              dic[j]+=1
        p=list(dic.values())
        pp.append(p)
    data[name]=pp

In [ ]:
vectorizer(df,'overview',dic_overview)

In [ ]:
vectorizer(df,'genres',dic_genres)

In [ ]:
df.head()

In [ ]:
dic_overview

In [ ]:
dic_genres

In [ ]:
X=df.overview.values
X=list(X)
y=df.genres.values
y=list(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
clf = MultiOutputClassifier(DecisionTreeClassifier()).fit(X_train, y_train)

In [ ]:
x=data[data['title']=='The Dark Knight Rises']
over=x.overview
for i in list(over.values)[0]:
    dic_overview[i]+=1
X=list(dic_overview.values())
# Let's predict the genre
ans=clf.predict([X])
ans=list(list(ans)[0])
print('The genres for the movie The dark knight rises are:')
print()
for i in range(len(ans)):
    if ans[i]==1:
        print(list(dic_genres.keys())[i])